in this news agency web site there is an "archive" page in which all published news are listed.in this page you can filter news by category and publish date.
if you open the link bellow you can see the archive page:

        http://www.entekhab.ir/fa/archive?service_id=18&sec_id=0&cat_id=0&rpp=100&from_date=1389/10/01&to_date=1397/01/30&p=1
        
there are 3 important keywords in the url :
service_id : specifies category of news
rpp : specifies number of news in each search page
p : page number

we use bs4 to parse web pages and urllib3 to send http requests.

In [4]:
from bs4 import BeautifulSoup
import urllib3

here we define a function to fetch an archive web page given page number and category of news(service_id)

In [5]:
def entekhab_get_page_for_category(page_number, service_id):
    ent_path = 'http://www.entekhab.ir/fa/archive?service_id={}&sec_id=0&cat_id=0&rpp=100&from_date=1389/10/01&to_date=1397/01/30&p={}'
    return ent_path.format(service_id,page_number)

the function bellow parses the output of the previous function and returns a list of news link.this function takes page number and news category(service_id) as inputs

In [6]:
def get_links_from_archive(page_number, service_id):
    try:
        http = urllib3.PoolManager()
        r = http.request('GET', entekhab_get_page_for_category(str(page_number),service_id))
        d = r.data.decode('utf-8')
        soup = BeautifulSoup(d,'lxml')
        r = soup.find_all('div',class_='news_archive_container')[0]
        r = r.find_all('div',class_='linear_news')
#         print(d)
        links = list()
        for i in r :
#             print(i)
    #         links.append('http://www.entekhab.ir/fa/news/'+i.find_all('a')[0]['href'].split('/')[3])
            links.append({'service_id' : service_id, 'url' :'http://www.entekhab.ir/fa/news/'+i.find_all('a')[0]['href'].split('/')[3]})
        return links
    except Exception as e:
        print(e)
        print("ERROR IN get_links_from_archive() page_number: " + str(page_number))
        return ['http://www.entekhab.ir']
    

In [9]:
ll = get_links_from_archive(12,service_id=2)
len(ll),ll[0]

(100, {'service_id': 2, 'url': 'http://www.entekhab.ir/fa/news/389088'})

the next thing to do is to fetch data of each page and store it in mongoDB.
"get_data_of_page" does this job . 

In [10]:
def get_data_of_page(page_url):
    try:
        http = urllib3.PoolManager()
        r = http.request('GET', page_url['url'])
        return {'url': page_url['url'],
                'content':r.data,
                'service_id':page_url['service_id']}
    except Exception as e:
        print(e)
        return {'url': page_url['url']
            ,'content':"PAGE_DATA_NOT_FETCHED_CURRECTLY",
               'service_id':-1}

"get_all_pages_data" function takes page number and service_id as inputs and thread number as an optional input and returns data of each news link in the archive page with news category = service_id

In [11]:
from multiprocessing.dummy import Pool as ThreadPool

In [12]:
def get_all_pages_data(page_number,service_id,thread_number=2):
    pool = ThreadPool(thread_number)
    results = pool.map(get_data_of_page,
                       get_links_from_archive(str(page_number),service_id=service_id))
    pool.close()
    pool.join()
    return results


# Performing action

the next thing is to run a for loop on the above function and store the output to mongoDB.
bellow is a sample output of each documnet in mongoDB:

        {'url': 'http://www.entekhab.ir/fa/news/389088',
         'content':"<html>...</html>",
         'service_id':2}
         
         
we collect news in these four categories

| category | service_id |
|----------|------------|
|politics|2|
|economics|5|
|havades|10|
|art and culture|18

In [12]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
db = client.sentimententekhab

we change service_id and run this loop again to collect news in different categories

In [25]:
for i in range(1,40):
    try:
        e=get_all_pages_data(page_number=i,service_id=18,thread_number=12)
        db.news.insert_many(e)
        print('inserted batch {}'.format(i))
    except Exception as e:
        print(e)
        print('error in '+str(i))

inserted batch 1
inserted batch 2
inserted batch 3
inserted batch 4
inserted batch 5
inserted batch 6
inserted batch 7
inserted batch 8
inserted batch 9
inserted batch 10
inserted batch 11
inserted batch 12
inserted batch 13
inserted batch 14
inserted batch 15
inserted batch 16
inserted batch 17
inserted batch 18
inserted batch 19
inserted batch 20
inserted batch 21
inserted batch 22
inserted batch 23
inserted batch 24
inserted batch 25
inserted batch 26
inserted batch 27
inserted batch 28
inserted batch 29
inserted batch 30
inserted batch 31
inserted batch 32
inserted batch 33
inserted batch 34
inserted batch 35
inserted batch 36
inserted batch 37
inserted batch 38
inserted batch 39
inserted batch 40
inserted batch 41
inserted batch 42
documents must be a non-empty list
error in 43
documents must be a non-empty list
error in 44
documents must be a non-empty list
error in 45
documents must be a non-empty list
error in 46
documents must be a non-empty list
error in 47
documents must be 

the next thing is to parse html data and store news data and corresponding service_id
so we define the news_parse function that takes html string as input and returns the text of news.

In [1]:
def news_parser (news_text):
    try:
        soup = BeautifulSoup(news_text, "lxml")
        ps = soup.find_all('div', {'class' : 'body col-xs-36'})[0]
        ps= ps.find_all('p')
        text=''
        for p in ps :
            text += p.get_text()
            text += ' | ' # use this character instead of new line
        
        table = {}
        table['text']=text
    except Exception as e:
        print('error in parsing : '+e)
        table = {'text':'\n'}
    return table

we parse the data and grab text and service_id of each news.

In [30]:
all_data_dictionary = {}
texts = list()
labels = list()

In [31]:
cnt=0
for i in db.news.find():
    cnt+=1
    if cnt % 1000 == 0:
        print(cnt)
    
    if i['content'] == "PAGE_DATA_NOT_FETCHED_CURRECTLY":
        print('error in db save ',i['url'])
    else:
        texts.append(new_parser(i['content'].decode('utf-8'))['text'])
        labels.append(i['service_id'])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


the last step in data preparation is storing data

In [34]:
all_data_dictionary = {'text':texts, 'label':labels}
df = pd.DataFrame(data=all_data_dictionary)
df.to_pickle('news_out')